<a href="https://www.kaggle.com/code/clemwo/understanding-trees-in-depth?scriptVersionId=196260587" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 51.8 MB/s eta 0:00:00


In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import Dash, html, dcc, callback, Output, Input

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification, load_iris, load_wine, load_diabetes
from sklearn.metrics import auc

import lightgbm
import xgboost
import xgboost as xgb
        
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/kaggle/input/drug-classification/drug200.csv


# 🤔Understanding Trees 🌳 In Depth💡

In this notebook we are going to investigate trained machine learning trees and their predictions in depth. For that matter we will use [Kaggle's Drug Classification](https://www.kaggle.com/datasets/prathamtripathi/drug-classification/data) dataset.

The dataset contains the following columns: 

| Column | Description | 
|:-------|:------------|
| `Age` | Age of the patient |
| `Sex` | Gender of the patient |
| `BP` | Blood pressure levels |
| `Cholesterol` | Cholesterol levels |
| `Na_to_K` | Sodium to potassium ratio in blood |
| `Drug` | Drug type |

In [3]:
drugs = pd.read_csv('/kaggle/input/drug-classification/drug200.csv')
print(f"Rows: {drugs.shape[0]:,}")
print(f"Cols: {drugs.shape[1]:,}")
drugs.head(3)

Rows: 200
Cols: 6


,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC


# Global Settings

In [4]:
RANDOM_STATE = 42

# Pre-Processing

In [5]:
features = ['Age', 'Na_to_K']
target = None

In [6]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
categorical_columns = drugs.select_dtypes(include=['object']).columns

for col in categorical_columns:
    encoded_col_name = col + '_Encoded'
    drugs[encoded_col_name] = label_encoder.fit_transform(drugs[col])
    
    if 'Drug' not in encoded_col_name:
        features.append(encoded_col_name)
    else:
        target = encoded_col_name

print(f"Modelling features: {features}")
print(f"Modelling target  : {target}")
drugs.head(3)

Modelling features: ['Age', 'Na_to_K', 'Sex_Encoded', 'BP_Encoded', 'Cholesterol_Encoded']
Modelling target  : Drug_Encoded


,Age,Sex,BP,Cholesterol,Na_to_K,Drug,Sex_Encoded,BP_Encoded,Cholesterol_Encoded,Drug_Encoded
0,23,F,HIGH,HIGH,25.355,DrugY,0,0,0,0
1,47,M,LOW,HIGH,13.093,drugC,1,1,0,3
2,47,M,LOW,HIGH,10.114,drugC,1,1,0,3


In [7]:
drugs['Drug_Encoded'].value_counts()

Drug_Encoded
0    91
4    54
1    23
3    16
2    16
Name: count, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(drugs[features], 
                                                    drugs[target], 
                                                    test_size=0.2, 
                                                    random_state=RANDOM_STATE)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (160, 5)
X_test shape: (40, 5)
y_train shape: (160,)
y_test shape: (40,)


In [9]:
xgb_params = {
    'max_depth': 3,                # maximum depth of each tree
    'eta': 0.3,                    # training step for each iteration
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',     # error evaluation for multiclass training
    'num_class': 5
}
num_round = 100
xgb_train = xgb.DMatrix(X_train, label = y_train, enable_categorical = False)
xgb_test = xgb.DMatrix(X_test, label = y_test, enable_categorical = False)

xgb_evals_result = {}
xgb_clf = xgb.train(xgb_params,
                    xgb_train, 
                    num_round, 
                    evals=[
                        (xgb_train, 'training') , 
                        (xgb_test, 'valid')
                    ], 
                    evals_result = xgb_evals_result)

[0]	training-mlogloss:1.04337	valid-mlogloss:1.10859
[1]	training-mlogloss:0.75492	valid-mlogloss:0.82417
[2]	training-mlogloss:0.56781	valid-mlogloss:0.64859
[3]	training-mlogloss:0.43791	valid-mlogloss:0.52975
[4]	training-mlogloss:0.33362	valid-mlogloss:0.42209
[5]	training-mlogloss:0.26289	valid-mlogloss:0.34346
[6]	training-mlogloss:0.20470	valid-mlogloss:0.28226
[7]	training-mlogloss:0.15942	valid-mlogloss:0.22560
[8]	training-mlogloss:0.12674	valid-mlogloss:0.19071
[9]	training-mlogloss:0.10159	valid-mlogloss:0.15636
[10]	training-mlogloss:0.08279	valid-mlogloss:0.13064
[11]	training-mlogloss:0.06861	valid-mlogloss:0.11463
[12]	training-mlogloss:0.05786	valid-mlogloss:0.09906
[13]	training-mlogloss:0.05015	valid-mlogloss:0.09140
[14]	training-mlogloss:0.04396	valid-mlogloss:0.08328
[15]	training-mlogloss:0.03921	valid-mlogloss:0.07778
[16]	training-mlogloss:0.03528	valid-mlogloss:0.07250
[17]	training-mlogloss:0.03206	valid-mlogloss:0.06781
[18]	training-mlogloss:0.02952	valid-m

# Model Complexity Tool

In [10]:
class BoostingComplexityAnalyzer:
    def __init__(self, model, classification_mode, metric_key, evals_result, n_classes = 2):
        self.model = model
        self.classification_mode = classification_mode
        self.n_classes = n_classes
        self.tree_details = None
        self.tree_depths = None
        self.cover_by_tree_index = None
        self.feature_gain_by_tree_index = None
        self.feature_thresholds_by_index = None
        self.tree_threshold_details = None
        self.unique_features = None
        self.tree_features = None
        self.metric_key = metric_key
        self.evals_result = evals_result
        self.learning_curves = None
        

    def fit(self):
        self._validate_inputs()
        self._extract_tree_details()
        self._calculate_tree_depths()
        self._compute_model_gain()
        self._compute_gain_attribution_by_feature()
        self._calculate_unique_splits()
        self._compute_min_max_threshold_analysis()
        self._compute_model_cover_attributions()
        self._compute_feature_usage_greediness_metric()

        if (self.metric_key and self.evals_result) is not None:
            self._compute_learning_curves()

    def _validate_inputs(self):
        """
        Validates the inputs to the class.

        Checks that the model is an instance of either `lightgbm.sklearn.LGBMClassifier`, `lightgbm.basic.Booster`, or `xgboost.core.Booster`. 
        It also checks that the `classification_mode` is either 'binary' or 'multi'. 
        If the `classification_mode` is 'multi', it further checks that `n_classes` is greater than 2.
        """
        assert type(self.model) in [
            lightgbm.sklearn.LGBMClassifier,
            lightgbm.basic.Booster,
            xgboost.core.Booster,
        ], "The input model must be either lightgbm.sklearn.LGBMClassifier or lightgbm.basic.Booster. XGBoost is not supported yet."

        assert self.classification_mode in [
            "binary",
            "multi",
        ], "classification_mode must be either 'binary' or 'multi'."

        if self.classification_mode == "multi":
            assert self.n_classes > 2, "Must provide n_classes > 2 for multi-class setting."

    def _extract_tree_details(self):
        """
        Extracts the details of each tree in the model and stores them in a DataFrame.

        This function checks the type of the model and calls the appropriate method to convert the model's trees to a DataFrame. 
        If the model is an XGBoost model, it renames the columns of the DataFrame to match the expected column names. 
        If the model is a multi-class classification model, it calculates the class and class tree index for each tree and adds these as new columns to the DataFrame.
        """
        if type(self.model) == lightgbm.sklearn.LGBMClassifier:
            self.tree_details = self.model.booster_.trees_to_dataframe()
        elif type(self.model) == lightgbm.basic.Booster:
            self.tree_details = self.model.trees_to_dataframe()
        if type(self.model) == xgboost.core.Booster:
            self.tree_details = self.model.trees_to_dataframe()

        print(self.tree_details.head())

        if type(self.model) in [xgboost.core.Booster]:
            self.tree_details = self.tree_details.rename(
                columns={
                    "Tree": "tree_index",
                    "Node": "node_depth",
                    "ID": "node_index",
                    "Feature": "split_feature",
                    "Split": "threshold",
                    "Gain": "split_gain",
                    "Cover": "count"
                }
            )

        if self.classification_mode == "multi":
            self.tree_details["tree_class"] = self.tree_details["tree_index"] % self.n_classes
            self.tree_details["class_tree_index"] = self.tree_details.groupby(["tree_class"])["tree_index"].rank(
                method = "dense"
            )

    def _calculate_tree_depths(self):
        """
        Calculates the depth of each tree in the model and computes various related metrics.

        Calculates the fraction of the total split gain that each split contributes. 
        It then calculates the maximum depth of each tree and the rolling average of the maximum depth over the last 5 trees. 
        If the 'split_feature_factorized' column does not exist in the tree details DataFrame, it factorizes the 'split_feature' column and adds the factorized values to the DataFrame. 
        It then computes the cumulative feature usage for each tree. 
        Finally, it calculates the total cover for each feature and tree index.
        """
        self.tree_details["frac_of_total_split_gain"] = self.tree_details.split_gain / self.tree_details.split_gain.sum()

        self.tree_depths = (
            self.tree_details.groupby("tree_index")["node_depth"]
            .max()
            .reset_index()
            .rename(columns = {"tree_index": "n_estimator", "node_depth": "max_node_depth"})
        )

        self.tree_depths["rolling_avg_max_depth"] = self.tree_depths["max_node_depth"].expanding(5).mean()

        if 'split_feature_factorized' not in self.tree_details.columns:
            self.tree_details['split_feature_factorized'], split_feature_encoding = pd.factorize(self.tree_details['split_feature'])
            split_feature_mapping = dict(enumerate(split_feature_encoding))

        self.tree_depths = self._compute_cumulative_feature_usage()
        self.cover = self.tree_details.groupby(['split_feature', 'tree_index'])['count'].sum().reset_index()
        self.leaf_analysis = self.tree_details[self.tree_details['split_feature'] == 'Leaf'].groupby(['tree_index'])['node_depth'].nunique().reset_index().rename(columns={'node_depth':'n_leaves_per_tree'})
        self.leaf_analysis['cumulative_leaves_in_model'] = self.leaf_analysis['n_leaves_per_tree'].cumsum()
        self.tree_depths = self.tree_depths.merge(self.leaf_analysis, how = 'left', left_on='n_estimator', right_on='tree_index')
        self.tree_depths['tree_depth_to_leaf_ratio'] = self.tree_depths['max_node_depth'] / self.tree_depths['n_leaves_per_tree']

    def _compute_model_cover_attributions(self):
        
        # Total Cover by Feature across entire model.
        self.cover_contribution = self.tree_details.groupby('split_feature')['count'].sum().sort_values(ascending = False).reset_index()
        self.median_cover = self.tree_details.groupby('split_feature')['count'].median().sort_values(ascending = False).reset_index()
        self.mean_cover = self.tree_details.groupby('split_feature')['count'].mean().sort_values(ascending = False).reset_index()
        self.std_dev_cover = self.tree_details.groupby('split_feature')['count'].std().sort_values(ascending = False).reset_index()

        n_features = self.cover_contribution.shape[0]

        # Compute Top N Features in Model by Cover from Sorted Index.
        self.cover_contribution = self.cover_contribution.reset_index().rename(columns = {'index':'top_n_features_in_model_by_cover'})
        self.cover_contribution['top_n_features_in_model_by_cover'] = self.cover_contribution['top_n_features_in_model_by_cover'] + 1

        self.median_cover = self.median_cover.reset_index().rename(columns = {'index':'top_n_features_in_model_by_median_cover'})
        self.median_cover['top_n_features_in_model_by_cover'] = self.median_cover['top_n_features_in_model_by_median_cover'] + 1

        self.mean_cover = self.mean_cover.reset_index().rename(columns = {'index':'top_n_features_in_model_by_mean_cover'})
        self.mean_cover['top_n_features_in_model_by_cover'] = self.mean_cover['top_n_features_in_model_by_mean_cover'] + 1

        self.std_dev_cover = self.std_dev_cover.reset_index().rename(columns = {'index':'top_n_features_in_model_by_std_dev_cover'})
        self.std_dev_cover['top_n_features_in_model_by_cover'] = self.std_dev_cover['top_n_features_in_model_by_std_dev_cover'] + 1

        # Compute Max Amount of Model Cover Seen.
        total_model_cover = self.cover_contribution['count'].cumsum().max()
        total_model_median_cover = self.median_cover['count'].cumsum().max()
        total_model_mean_cover = self.mean_cover['count'].cumsum().max()
        total_model_std_dev_cover = self.std_dev_cover['count'].cumsum().max()

        # Compute Cumulative Fraction of Model Cover.
        self.cover_contribution['cumulative_total_cover_over_features'] = (self.cover_contribution['count'].cumsum() / total_model_cover).apply(lambda x: round(x, 3))
        self.median_cover['cumulative_median_cover_over_features'] = (self.median_cover['count'].cumsum() / total_model_median_cover).apply(lambda x: round(x, 3))
        self.mean_cover['cumulative_mean_cover_over_features'] = (self.mean_cover['count'].cumsum() / total_model_mean_cover).apply(lambda x: round(x, 3))
        self.std_dev_cover['cumulative_std_dev_cover_over_features'] = (self.std_dev_cover['count'].cumsum() / total_model_std_dev_cover).apply(lambda x: round(x, 3))

        # Compute Fraction of Model Features.
        self.cover_contribution['fraction_of_features'] = (self.cover_contribution['top_n_features_in_model_by_cover'] / n_features).apply(lambda x: round(x, 3))
        self.median_cover['fraction_of_features'] = (self.median_cover['top_n_features_in_model_by_cover'] / n_features).apply(lambda x: round(x, 3))
        self.mean_cover['fraction_of_features'] = (self.mean_cover['top_n_features_in_model_by_cover'] / n_features).apply(lambda x: round(x, 3))
        self.std_dev_cover['fraction_of_features'] = (self.std_dev_cover['top_n_features_in_model_by_cover'] / n_features).apply(lambda x: round(x, 3))

        # Merge all together on `top_n_features_in_model_by_cover`.
        self.cover_contribution = self.cover_contribution.merge(self.std_dev_cover[['top_n_features_in_model_by_cover', 'cumulative_std_dev_cover_over_features']], 
                                                            how = 'left', 
                                                            left_on = 'top_n_features_in_model_by_cover',
                                                            right_on = 'top_n_features_in_model_by_cover'
                                                            )

        self.cover_contribution = self.cover_contribution.merge(self.median_cover[['top_n_features_in_model_by_cover', 'cumulative_median_cover_over_features']], 
                                                            how = 'left', 
                                                            left_on = 'top_n_features_in_model_by_cover',
                                                            right_on = 'top_n_features_in_model_by_cover'
                                                            )

        self.cover_contribution = self.cover_contribution.merge(self.mean_cover[['top_n_features_in_model_by_cover', 'cumulative_mean_cover_over_features']], 
                                                            how = 'left', 
                                                            left_on = 'top_n_features_in_model_by_cover',
                                                            right_on = 'top_n_features_in_model_by_cover'
                                                            )
        

    def _compute_model_gain(self):
        """
        Computes the total gain of the model and the cumulative gain for each tree.

        Calculates the total gain of the model by summing the split gains of all trees. 
        It also calculates the cumulative gain for each tree by summing the split gains of the current tree and all previous trees. 
        Finally, it calculates the cumulative fraction of the total model gain that each tree contributes.
        """
        self.model_gain_by_tree_index = self.tree_details.groupby(["tree_index"])["split_gain"].sum().reset_index()
        self.total_model_gain = self.model_gain_by_tree_index['split_gain'].sum()
        self.model_gain_by_tree_index['cumulative_model_gain_by_tree_index'] = self.model_gain_by_tree_index['split_gain'].cumsum()
        self.model_gain_by_tree_index['cumulative_fraction_of_model_gain'] = self.model_gain_by_tree_index['cumulative_model_gain_by_tree_index'] / self.total_model_gain


    def _compute_cumulative_feature_usage(self):
        """
        Computes the cumulative number of unique features used per tree_index in a Gradient Boost model object.

        In the binary classification context, the tree_index represents the entire boosting round. 
        In the multiclass context, the tree_index represents a One-vs-All (OVA) tree for a particular class, sorted by modulo(class). 
        In this context, `tree_index` is adjusted to represent the concept of the boosting round/ estimator only for the 
        particular class the tree votes for. 

        Iterates over each tree in the model. For each tree, it adds the features used by the tree to a set of features used by that tree. 
        It also adds the features used by the tree to a global set of features used by all trees seen so far. 
        It then calculates the number of unique features used by the tree and the cumulative number of unique features used by all trees seen so far. 
        Finally, it adds these counts to the tree details DataFrame.

        """

        tree_idx_keys = self.tree_details.tree_index.unique()
        self.unique_features = set()
        self.tree_features = {key: {"features_used": []} for key in tree_idx_keys}

        for row_idx, node_row in self.tree_details.iterrows():
            if node_row["split_feature"] is not None:
                self.tree_features[node_row["tree_index"]]["features_used"].append(node_row["split_feature"])

        for tree_index, features_dict in self.tree_features.items():
            self.tree_features[tree_index]["features_used"] = set(features_dict["features_used"])
            self.tree_features[tree_index]["n_features_used"] = len(features_dict["features_used"])

            for feature in features_dict["features_used"]:
                self.unique_features.add(feature)

            self.tree_features[tree_index]["cumulative_features_used"] = self.unique_features
            self.tree_features[tree_index]["n_cumulative_features_used"] = len(
                self.tree_features[tree_index]["cumulative_features_used"]
            )

        self.tree_depths["n_cumulative_features_used"] = pd.DataFrame(self.tree_features).transpose()[
            "n_cumulative_features_used"
        ]

        return self.tree_depths

    def _compute_feature_usage_greediness_metric(self):
            model_area = auc(self.tree_depths['n_estimator'], self.tree_depths['n_cumulative_features_used'])

            max_features = self.tree_depths['n_cumulative_features_used'].max()
            max_estimators = self.tree_depths['n_estimator'].max()

            max_area = max_features * max_estimators
            reference_area = 0.5 * (max_area)
            self.greediness_metric = (max_area - model_area) / max_area

            if self.greediness_metric < 0.25:
                self.user_suggestion = f'Greediness metric in [0.0, 0.25) - suggests model uses all features early on in boosting rounds, check for overfitting / extraneous splits on noise.'
            elif self.greediness_metric > 0.75:
                self.user_suggestion = f'Greediness metric in (0.75, 1.0] - suggests model uses all features later in boosting rounds, check for bias / underfitting.'
            else:
                self.user_suggestion = f'Greediness metric between 0.25 and 0.75 - suggests model uses adds new features slowly over boosting rounds.'

    def _calculate_unique_splits(self):
        self.tree_details_grouped = self.tree_details_numeric_features.groupby("split_feature_factorized").apply(self.count_expanding_unique_thresholds)
        self.tree_threshold_details = self.tree_details_grouped.reset_index(drop = True)

    def _compute_gain_attribution_by_feature(self):
        """Computes the total gain attributable to a particular feature in a Gradient 
        Boosting model object across each successive boosting round."""

        self.feature_gain_by_tree_index = (
            self.tree_details.groupby(["split_feature", "tree_index"])["split_gain"].sum().reset_index()
        )

        self.feature_gain_by_tree_index = pd.concat(
            [
                self.feature_gain_by_tree_index,
                self.tree_details.groupby(["split_feature", "tree_index"])["frac_of_total_split_gain"]
                .sum()
                .reset_index()["frac_of_total_split_gain"],
            ],
            axis=1,
        )

        self.feature_gain_by_tree_index["cumulative_frac_total_split_gain"] = self.feature_gain_by_tree_index.groupby(
            "split_feature"
        )["frac_of_total_split_gain"].cumsum()

        # Select records with numeric threshold otherwise std() will fail.
        self.tree_details_numeric_features = self.tree_details.copy()
        self.tree_details_numeric_features["threshold"] = pd.to_numeric(self.tree_details_numeric_features["threshold"], errors="coerce")
        self.tree_details_numeric_features.dropna(subset=["threshold"], inplace=True)

        self.feature_thresholds_by_index = (
            self.tree_details_numeric_features.groupby(["split_feature", "tree_index"])["threshold"]
            .expanding()
            .std()
            .reset_index()
            .rename(columns = {"threshold": "sigma_in_threshold_up_to_current_estimator"})
        )

        self.feature_thresholds_by_tree = self.feature_thresholds_by_index.groupby(['split_feature', 'tree_index'])['sigma_in_threshold_up_to_current_estimator'].last().reset_index()

        # TODO: Compute Total Gain by Feature across entire model
        self.feature_gains = self.feature_gain_by_tree_index.groupby('split_feature')['split_gain'].sum().sort_values(ascending = False).reset_index()
        self.n_features = self.feature_gains.shape[0]
        self.feature_gains = self.feature_gains.reset_index().rename(columns = {'index':'top_n_features_in_model_by_gain'})
        self.feature_gains['top_n_features_in_model_by_gain'] = self.feature_gains['top_n_features_in_model_by_gain'] + 1
        self.total_model_gain = self.feature_gains['split_gain'].cumsum().max()
        self.feature_gains['cumulative_gain_over_features'] = (self.feature_gains['split_gain'].cumsum() / self.total_model_gain).apply(lambda x: round(x, 3))
        self.feature_gains['fraction_of_features'] = (self.feature_gains['top_n_features_in_model_by_gain'] / self.n_features).apply(lambda x: round(x, 3))

    def _compute_min_max_threshold_analysis(self):
        """
        Computes the min-max scaled range of threshold values for each feature and updates the tree details DataFrame.

        First creates a copy of the tree details DataFrame and converts the threshold column to numeric. 
        It then calculates the min-max scaled range of threshold values for each feature, which is the 
        difference between each threshold value and the minimum threshold value for the feature, 
        divided by the range of threshold values for the feature.

        Then merges the calculated min-max scaled ranges back into the tree details DataFrame. 
        It then calculates the maximum min-max scaled range for each feature and tree index, 
        and merges these maximum values into the feature thresholds by tree DataFrame.
        """
        self.tree_details_df_numeric = self.tree_details.copy()
        self.tree_details_df_numeric["threshold"] = pd.to_numeric(self.tree_details_df_numeric["threshold"], errors = "coerce")

        self.min_max_feature_ranges = self.tree_details_df_numeric.groupby("split_feature_factorized")['threshold'].apply(lambda series: (series - series.min()) / (series.max() - series.min()))
        self.min_max_feature_ranges = self.min_max_feature_ranges.reset_index().set_index('level_1').rename(columns={'threshold': 'min_max_scaled_range_in_threshold'})

        self.tree_details_df_numeric = self.tree_details_df_numeric.merge(self.min_max_feature_ranges['min_max_scaled_range_in_threshold'], how = 'left', left_index = True, right_on = 'level_1')

        self.min_max_by_tree_index = self.tree_details_df_numeric.groupby(['split_feature_factorized', 'tree_index'])['min_max_scaled_range_in_threshold'].max().reset_index()

        self.feature_thresholds_by_tree = self.feature_thresholds_by_tree.merge(self.min_max_by_tree_index['min_max_scaled_range_in_threshold'], how = 'left', left_index = True, right_index = True)


    def _compute_learning_curves(self):
        """
        Computes the model learning curves across training and validation in accordance with different metrics tracked during the model's training.

        This function computes the learning curves. 
        It extracts the evaluation results for the training and validation datasets, 
        and computes the difference and the first derivative of the difference between the training and validation losses. 
        It also computes a rolling average of the first derivative of the difference.
        """

        self.iterations = []

        self.metric_keys = self.evals_result["training"].keys()
        assert (
            self.metric_key in self.evals_result["training"].keys()
        ), f'The metric key specified was not found in logged metrics of the training dataset.\nTry "logloss" or "mlogloss" for XGBoost models.\nmetric_key provided: {self.metric_key} but results have {self.metric_keys}'

        for key, item in self.evals_result.items():
            self.iterations.append(pd.Series(item[self.metric_key], name = key))

        # Extract metric data.
        self.learning_curves = pd.DataFrame(self.iterations).transpose()
        self.learning_curves = self.learning_curves.reset_index().rename(columns = {"index": "n_estimators"})

        # Compute additional profiles: differential and first derivative of differential.
        self.learning_curves["train_val_loss_differential"] = self.learning_curves.iloc[:, 1] - self.learning_curves.iloc[:, 2]
        self.learning_curves["d1_train_val_loss"] = self.learning_curves["train_val_loss_differential"].diff()
        self.learning_curves["d1_train_val_loss_rolling5_avg"] = self.learning_curves["d1_train_val_loss"].rolling(window = 5).mean()

        # return self.learning_curves


    def count_expanding_unique_thresholds(self, group):
        """
        Computes the model learning curves across training and validation in accordance with different metrics tracked during the model's training.
        It extracts the evaluation results for the training and validation datasets, 
        and computes the difference and the first derivative of the difference between the training and validation losses. 
        It also computes a rolling average of the first derivative of the difference.
        """
        group["n_unique_thresholds_seen"] = (
            group["threshold"]
            .expanding()
            .apply(lambda vals: len(np.unique(vals)))
            .rename("n_unique_thresholds_seen")
        )
        return group


    def count_expanding_unique_features(self, group):
        """Counts the number of unique features seen in an expanding window of the input group."""
        group["n_cumulative_features_seen"] = (
            group["split_feature"]
            .expanding()
            .apply(lambda vals: len(np.unique(vals)))
            .rename("n_cumulative_features_seen")
        )
        return group


    def transform(self):
        return dict(
            tree_details = self.tree_details,
            tree_depths = self.tree_depths,
            feature_gain_by_tree_index = self.feature_gain_by_tree_index,
            feature_thresholds_by_index = self.feature_thresholds_by_index,
            feature_thresholds_by_tree = self.feature_thresholds_by_tree,
            model_gain_by_tree_index = self.model_gain_by_tree_index,
            gain_contribution = self.feature_gains,
            tree_threshold_details = self.tree_threshold_details,
            learning_curves = self.learning_curves,
            cover = self.cover,
            cover_contribution = self.cover_contribution,
            feature_greediness = (self.greediness_metric, self.user_suggestion)
        )

In [11]:
tca = BoostingComplexityAnalyzer(model=xgb_clf, 
                                 classification_mode='binary', 
                                 n_classes=5, 
                                 metric_key='mlogloss', 
                                 evals_result=xgb_evals_result)
tca.fit()
tree_complexity_dict = tca.transform()

   Tree  Node   ID     Feature   Split  Yes   No Missing        Gain  \
0     0     0  0-0     Na_to_K  15.015  0-1  0-2     0-2  119.032104   
1     0     1  0-1        Leaf     NaN  NaN  NaN     NaN   -0.180775   
2     0     2  0-2        Leaf     NaN  NaN  NaN     NaN    0.720379   
3     1     0  1-0  BP_Encoded   1.000  1-1  1-2     1-2    8.576456   
4     1     1  1-1     Na_to_K  13.093  1-3  1-4     1-4   16.746288   

       Cover  Category  
0  51.200005       NaN  
1  26.880001       NaN  
2  24.320002       NaN  
3  51.200005       NaN  
4  19.840002       NaN  


/tmp/ipykernel_17/645560555.py:258: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  self.tree_details_grouped = self.tree_details_numeric_features.groupby("split_feature_factorized").apply(self.count_expanding_unique_thresholds)


## Plot the Max Depth by Boosting Round

In [12]:
class Plotter:
    def __init__(self, plotting_data):
        self.plot_data = plotting_data
        self.subset_threshold_df_list = []

    def plot(self, df_key, x, y, color, title, log_y=False):
        fig = px.line(data_frame=self.plot_data[df_key], x=x, y=y, color=color, log_y=log_y, title=title)
        self._customize_plot(fig)
        return fig

    def plot_feature_cover_distribution(self):
        return self.plot('cover', 'tree_index', 'count', 'split_feature', 'Total Cover by Feature', log_y=True)

    def plot_cover_by_feature(self):
        return self.plot('cover', 'tree_index', 'count', 'split_feature', 'Cover by Feature')

    def _customize_plot(self, fig):
        fig.update_layout(plot_bgcolor='#FFFFFF')

    def _add_deselect_all_button(self, fig):
        fig.update_layout(dict(updatemenus=[
            dict(type="buttons", buttons=[
                dict(args=["visible", "legendonly"], label="Deselect All", method="restyle"),
                dict(args=["visible", True], label="Select All", method="restyle")
            ])
        ]))

    def return_tab_content_list(self):
        return [dcc.Tab(label=f'Tab {i}', children=[dcc.Graph(figure=self.plot_feature_cover_distribution())]) for i in range(4)]

    def plot_expanding_tree_depth_by_estimator(self):
        """
        Plots the tree depth by number of estimators. 
        This function takes a DataFrame plot_data['tree_depths'] as input, 
        creates a scatter plot using Plotly, customizes the layout and axes of the plot, and then displays the plot.
        """
        fig = go.Figure(go.Scatter(x = self.plot_data['tree_depths']['n_estimator'], 
                                   y = self.plot_data['tree_depths']['rolling_avg_max_depth']))
        fig.update_layout(
            title = "Rolling Avg. Tree Depth by n_estimators / boosting rounds",
            xaxis_title = "n_estimators",
            yaxis_title = "rolling_avg_max_depth",
        )

        self._customize_plot(fig)

        return fig  
    
    def plot_cumulative_features_used_by_estimator(self):
        """
        Plots the cumulative features used by number of estimators. 
        This function takes a DataFrame plot_data['tree_depths'] as input, 
        creates a scatter plot using Plotly, customizes the layout and axes of the plot, and then displays the plot.
        """
        fig = go.Figure(go.Scatter(x = self.plot_data['tree_depths']['n_estimator'], 
                                   y = self.plot_data['tree_depths']['n_cumulative_features_used']))
        
        # Determine Max y_value.
        y_max = self.plot_data['tree_depths']['n_cumulative_features_used'].max()
        x_max = self.plot_data['tree_depths']['n_estimator'].max()

        # Compute Greediness Metric and Warning.
        greediness_metric, user_suggestion = self.plot_data['feature_greediness'][0], self.plot_data['feature_greediness'][1]

        # Add Diagonal Reference Line.
        fig.update_layout(shapes = [{'type': 'line', 
                                     'y0': 0, 
                                     'y1': y_max, 
                                     'x0': 0, 
                                     'x1': x_max, 
                                     'line_dash': 'dot',
                                     'label' : dict(text = 'Reference Line: 1 New Feature per Boosting Round')
                                    #  'annotation_position' : 'top right'
                                     }])

        fig.update_layout(
            title = f"Cumulative Features used by n_estimators / boosting rounds<br>Feature Greediness Metric: {round(greediness_metric, 2)}<br>Warning: {user_suggestion}",
            xaxis_title = "n_estimators",
            yaxis_title = "n_cumulative_features_used"
        )

        self._customize_plot(fig)

        return fig    
    
    def plot_relative_gain_attribution_by_feature(self):
        """
        Plots the relative attribution of total model gain by feature. 
        This function takes a DataFrame plot_data['feature_gain_by_tree_index'] as input, 
        creates a line plot using Plotly Express, customizes the layout and axes of the plot, and then displays the plot.
        """
        fig = px.line(data_frame = self.plot_data['feature_gain_by_tree_index'], 
                    x = "tree_index", 
                    y = "frac_of_total_split_gain", 
                    color = 'split_feature',
                    title = 'Relative Attribution of Total Model Gain by Feature<br><sup>Ordered by No. of Boosting Rounds</br></sup>')

        self._customize_plot(fig)
        self._add_deselect_all_button(fig)

        return fig    

    def plot_cumulative_model_gain_by_estimator(self):
        """
        Plots the cumulative model gain by number of boosting rounds. 
        This function takes a DataFrame model_gain_by_tree_index as input, 
        creates a line plot using Plotly Express, customizes the layout and axes of the plot, 
        adds reference lines at 80% and 90% of model gain, and then displays the plot.
        """
        fig = px.line(data_frame = self.plot_data['model_gain_by_tree_index'], 
                    x = "tree_index", 
                    y = "cumulative_fraction_of_model_gain", 
                    title = 'Cumulative Gain Attribution by No. of Boosting Rounds')

        self._customize_plot(fig)

        # Add Horizontal Reference Line
        fig.add_hline(y = 0.90, 
                    line_dash = "dot", 
                    annotation_text = "90% of Model Gain Reference", 
                    annotation_position = "bottom right")

        fig.add_hline(y = 0.80, 
                    line_dash = "dot", 
                    annotation_text = "80% of Model Gain Reference", 
                    annotation_position = "bottom right")

        return fig   
    
    def plot_ecdf_gain_attribution_by_feature(self):
        """
        Plots the ECDF of cumulative gain attribution by feature. 
        This function takes a DataFrame plot_data['feature_gain_by_tree_index'] as input, 
        creates an ECDF plot using Plotly Express, customizes the layout and axes of the plot, 
        adds a menu for selecting/deselecting all features, and then displays the plot.
        """

        fig = px.ecdf(data_frame = self.plot_data['feature_gain_by_tree_index'], 
                      x = "tree_index", 
                      y = "cumulative_frac_total_split_gain", 
                      color = 'split_feature',
                      title = 'ECDF: Cumulative Gain Attribution by Feature<br><sup>Ordered by No. of Boosting Rounds</br></sup>')
        
        self._customize_plot(fig)
        self._add_deselect_all_button(fig)

        return fig
    
    def plot_cover_by_feature(self):
        """
        Plots the distribution of cover along the tree_index by feature. 
        This function takes a DataFrame tree_details_ as input, creates a line plot using Plotly Express, 
        customizes the layout and axes of the plot, and then displays the plot.
        """

        fig = px.line(data_frame = self.plot_data['cover'], 
                      x = "tree_index", 
                      y = "count", 
                      color = 'split_feature',
                      title = 'Cover by Feature<br><sup>Ordered by No. of Boosting Rounds</br></sup>')
        
        self._customize_plot(fig)
        self._add_deselect_all_button(fig)

        return fig    
    
    def plot_rolling_threshold_sigma_by_estimator(self):
        """Plots the standard deviation of split thresholds by feature over the number of boosting rounds."""
        fig = px.line(data_frame = self.plot_data['feature_thresholds_by_tree'],
                    x = 'tree_index',
                    y = 'sigma_in_threshold_up_to_current_estimator',
                    color = 'split_feature',
                    title = 'Standard Deviation of Split Thresholds by Feature<br><sup>Ordered by No. of Boosting Rounds')

        self._customize_plot(fig)
        self._add_deselect_all_button(fig)

        return fig
    
    def plot_threshold_min_max_range_by_estimator(self):
        """Plots the min-max range of split thresholds by feature over the number of boosting rounds."""
        fig = px.line(data_frame = self.plot_data['feature_thresholds_by_tree'], 
                        x = "tree_index", 
                        y = "min_max_scaled_range_in_threshold", 
                        color = 'split_feature',
                    #  line_shape = 'linear',
                        title = 'Min-Max Range of Split Thresholds by Feature<br><sup>Ordered by No. of Boosting Rounds')

        self._customize_plot(fig)
        self._add_deselect_all_button(fig)

        return fig   
    
    def plot_new_unique_splits_by_feature(self):
        """Plots the cumulative number of unique split thresholds by feature over the number of boosting rounds."""
        fig = px.line(self.plot_data['tree_threshold_details'], 
            x = 'tree_index', 
            y = 'n_unique_thresholds_seen', 
            color = 'split_feature', 
            markers = True,
            title = 'Cumulative No. of Unique Thresholds by Estimator<br><sup>tree_index represents One-vs-All (OVA) Tree Index in a multi-class setting.')

        self._customize_plot(fig)
        self._add_deselect_all_button(fig)

        return fig        

In [13]:
plotter = Plotter(plotting_data=tree_complexity_dict)
plotter.plot_data.keys()

dict_keys(['tree_details', 'tree_depths', 'feature_gain_by_tree_index', 'feature_thresholds_by_index', 'feature_thresholds_by_tree', 'model_gain_by_tree_index', 'gain_contribution', 'tree_threshold_details', 'learning_curves', 'cover', 'cover_contribution', 'feature_greediness'])

In [14]:
plotter.plot_feature_cover_distribution()

In [15]:
plotter.plot_cover_by_feature()

## Rolling Avg. Max Depth by Boosting Round

In [16]:
plotter.plot_expanding_tree_depth_by_estimator()

## Plot the Cumulative Number of Features by Boosting Round

In [17]:
plotter.plot_cumulative_features_used_by_estimator()

## Feature Gain Analysis

In [18]:
plotter.plot_relative_gain_attribution_by_feature()

## Cumulative Fraction of Total Gain by Tree Index

In [19]:
plotter.plot_cumulative_model_gain_by_estimator()

## Number of features (backwards recursively eliminated) that sum to a percentage of a total model gain

In [20]:
plotter.plot_ecdf_gain_attribution_by_feature()

## Split Threshold Analysis

In [21]:
plotter.plot_rolling_threshold_sigma_by_estimator()

In [22]:
plotter.plot_threshold_min_max_range_by_estimator()

## Number of `new_unique_splits` per boosting round per feature

In [23]:
plotter.plot_new_unique_splits_by_feature()

---

💚 Thank you for reading 💚

If you have any questions or feedback, feel free to leave a comment 🤔

This notebook is __still in progress__.

Please __UPVOTE__ if you enjoyed this notebook 🙏